Setup the kernel

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import downscale_local_mean
from os.path import join
from tqdm.notebook import tqdm
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

C:\Users\Zhiqing\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Zhiqing\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Zhiqing\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Zhiqing\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\frame

In [17]:
input_folder = 'G:/Github/Caravan_challenge/data'

df_mask = pd.read_csv(join(input_folder, 'train_masks.csv'), usecols=['img'])

ids_train = df_mask['img'].map(lambda s: s.split('_')[0]).unique()

imgs_idx = list(range(1, 17))

img_resize = 2
img_1 = 1280
img_2 = 1918
img_chan = 3

Define helper functions for later

In [9]:
load_img = lambda im, idx: imread(join(input_folder, 'train', '{}_{:02d}.jpg'.format(im, idx)))
load_mask = lambda im, idx: imread(join(input_folder, 'train_masks', '{}_{:02d}_mask.gif'.format(im, idx)))
resize = lambda im: downscale_local_mean(im, (img_resize,img_resize) if im.ndim==2 else (img_resize,img_resize,1))
mask_image = lambda im, mask: (im * np.expand_dims(mask, 2))

In [30]:
num_train = 120  # len(ids_train)

# Load data for position id=1
X = np.empty((num_train, img_1//img_resize,img_2//img_resize , 9), dtype=np.float32)
y = np.empty((num_train, img_1//img_resize, img_2//img_resize, 9), dtype=np.float32)


idx = 1 # Rotation index
for i, img_id in enumerate(ids_train[:num_train]):
    imgs_id = [resize(load_img(img_id, j)) for j in imgs_idx]
    # Input is image + mean image per channel + std image per channel
    X[i, ..., :9] = np.concatenate([imgs_id[idx-1], np.mean(imgs_id, axis=0), np.std(imgs_id, axis=0)], axis=2)
    y[i] = resize(np.expand_dims(load_mask(img_id, idx), 2)) / 255.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Normalize input and output
X_mean = X_train.mean(axis=(0,1,2), keepdims=True)
X_std = X_train.std(axis=(0,1,2), keepdims=True)

X_train -= X_mean
X_train /= X_std

X_val -= X_mean
X_val /= X_std

Set up a simple CNN model

In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import Sequential

model = Sequential()
model.add( Conv2D(16, 3, activation='relu', padding='same', input_shape=(img_1//img_resize, img_2//img_resize, 9)) )
model.add( Conv2D(32, 3, activation='relu', padding='same') )
model.add( Conv2D(1, 5, activation='sigmoid', padding='same') )

Complile model

In [ ]:
model.compile(optimizer = 'Adam', loss = 'BinaryCrossentropy', metrics=['accuracy', 'Precision'])